In [1]:
library(tidyverse)
library(broom)
library(repr)
library(digest)
library(infer)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.6      v purrr   0.3.4 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.0      v stringr 1.4.1 
v readr   2.1.2      v forcats 0.5.2 

Warning message:
"package 'ggplot2' was built under R version 4.1.3"
Warning message:
"package 'tibble' was built under R version 4.1.3"
Warning message:
"package 'tidyr' was built under R version 4.1.2"
Warning message:
"package 'readr' was built under R version 4.1.2"
Warning message:
"package 'dplyr' was built under R version 4.1.3"
Warning message:
"package 'stringr' was built under R version 4.1.3"
Warning message:
"package 'forcats' was built under R version 4.1.3"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Warning message:
"package 'broom' was built under 

In [2]:
options(repr.plot.width=15, repr.plot.height=7)

In [5]:
employment <- read_csv("https://raw.githubusercontent.com/GWilliam02/STAT201-Project/main/data/codebook.csv")
head(employment)

Rows: 397 Columns: 10
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (7): Variable_Variable, EnglishLabel_EtiquetteAnglais, FrenchLabel_Etiqu...
dbl (3): Field_Champ, Position_Position, Length_Longueur

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Field_Champ,Position_Position,Length_Longueur,Variable_Variable,EnglishLabel_EtiquetteAnglais,FrenchLabel_EtiquetteFrancais,EnglishUniverse_UniversAnglais,FrenchUniverse_UniversFrancais,EnglishNote_NoteAnglais,FrenchNote_NoteFrancais
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,7,rec_num,Order of record in file,Ordre de l'observation dans le fichier,All respondents,Tous les r<e9>pondantes,NA,NA
NA,NA,NA,1-9999999,NA,NA,NA,NA,NA,NA
2,8,4,survyear,Survey year,Ann<e9>e d'enqu<ea>te,All respondents,Tous les r<e9>pondantes,NA,NA
NA,NA,NA,1976-,NA,NA,NA,NA,NA,NA
3,12,2,survmnth,Survey month,Mois d'enqu<ea>te,All respondents,Tous les r<e9>pondantes,NA,NA
NA,NA,NA,01,January,Janvier,NA,NA,NA,NA
